In [1]:
from aiida import load_profile  
load_profile()

Profile<uuid='1ad5c4ff2c1141ee9b4a511ac6859016' name='presto'>

In [2]:
from aiida.orm import load_code
from aiida_mlip.data.model import ModelData

uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp")

janus_code = load_code("janus@localhost")
qe_code = load_code("qe@localhost")

In [3]:
from aiida.plugins import CalculationFactory
descriptorsCalc = CalculationFactory("mlip.descriptors")

In [4]:
from aiida_workgraph import WorkGraph, task
from pathlib import Path
from ase.io import read, write, iread
from ase import Atoms
import numpy as np
from aiida.orm import SinglefileData, Float, InstalledCode, List, Dict, KpointsData, StructureData, load_group, Str, Bool, Int
from aiida_quantumespresso.calculations.pw import PwCalculation
from aiida_workgraph.manager import get_current_graph
from ase import units
import tempfile
from pathlib import Path
from sample_split import process_and_split_data

@task.calcfunction(outputs=["scaled_file"])
def create_scales(
    min_v: Float,
    max_v:Float,
    num_structs: int,
    **structures
): 
    
    atoms = []
    for i, struct in structures.items():
        with struct.as_path() as path:
            atoms.append(read(path))

    lattice_scalars = np.cbrt(np.linspace(min_v.value, max_v.value, num_structs.value))
    b = atoms.copy()
    for i, s in enumerate(lattice_scalars):
        b[i].set_cell((atoms[i].get_cell()) * s, scale_atoms=True)
        write("scaled.extxyz",b[i],append=i>0)

    return {
        "scaled_file": SinglefileData(Path("scaled.extxyz").resolve())
    }

@task.graph(outputs = ["structures"])
def qe(
    code: InstalledCode,
    kpoints_mesh: List,
    task_metadata: Dict,
    scaled_file: SinglefileData,
    ):

    wg = get_current_graph()

    kpoints = KpointsData()
    kpoints.set_kpoints_mesh(kpoints_mesh)

    pseudo_family = load_group('SSSP/1.3/PBE/efficiency')
    
    output_structures = {}

    with scaled_file.as_path() as path:
        for i, structs in enumerate(iread(path, format="extxyz")):
            
            structure = StructureData(ase=structs)
            pseudos = pseudo_family.get_pseudos(structure=structure)

            ecutwfc, ecutrho = pseudo_family.get_recommended_cutoffs(
                structure=structure,
                unit='Ry',
            )

            pw_params = {
                "CONTROL": {
                    "calculation": "scf",
                    'tprnfor': True,
                    'tstress': True,
                },
                "SYSTEM": {
                    "ecutwfc": ecutwfc,
                    "ecutrho": ecutrho,
                },
            }
            
            qe_task = wg.add_task(
                PwCalculation,
                code=code,
                parameters=pw_params,
                kpoints=kpoints,
                pseudos=pseudos,
                metadata=task_metadata.value,
                structure=structure,
            )

            output_structures[f"struct{i}"] = {
                    "trajectory":qe_task.outputs.output_trajectory,
                    "parameters": qe_task.outputs.output_parameters
                }
        
        wg.update_ctx({
            "structures": output_structures
        })
            # wg.update_ctx({
            #     f"struct{i}" :{
            #         "trajectory":qe_task.outputs.output_trajectory,
            #         "parameters": qe_task.outputs.output_parameters
            #     }
            # })

    return {
        "structures": wg.ctx.structures,
    }

@task.calcfunction(outputs=["test_file"])
def create_train_files(**structures):

    tmpfile = tempfile.NamedTemporaryFile(suffix=".extxyz")
    
    for structs in structures.values():    

        trajectory = structs["trajectory"]

        fileStructure = trajectory.get_structure(index=0)
        fileAtoms = fileStructure.get_ase()

        stress = trajectory.arrays["stress"][0]
        converted_stress = stress * units.GPa
        fileAtoms.info["qe_stress"] = converted_stress

        fileAtoms.info["units"] = {"energy": "eV","forces": "ev/Ang","stress": "ev/Ang^3"}
        fileAtoms.set_array("qe_forces", trajectory.arrays["forces"][0])

        parameters = structs["parameters"]
        fileParams = parameters.get_dict()
        fileAtoms.info["qe_energy"] = fileParams["energy"]
        write(Path(tmpfile.name), fileAtoms, append=True)

    process_inputs = {
        "config_types": Str(""),
        "prefix": Str(""),
        "scale": Float(1.0e5),
        "append_mode": Bool(False),
        "n_samples": Int(len(structures)),
        "trajectory_data": tmpfile.name
    }

    process_and_split_data(**process_inputs)
                
    return{
        "test_file": SinglefileData(tmpfile)
    }


In [5]:
calc_inputs = {
    "code": janus_code,
    "model": model,
    "arch": Str(model.architecture),
    "device": Str("cuda"),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
}

scales_inputs = {
    "min_v": 0.95,
    "max_v": 1.05,
    "num_structs": 12
}

qe_inputs = {
    "task_metadata": Dict({
            "options": {
                "resources": {
                    "tot_num_mpiprocs":1,
                    'num_mpiprocs_per_machine':1,
                    'num_cores_per_mpiproc':8,
                },
                "max_wallclock_seconds": 3600,
                "queue_name": "scarf",
                "qos": "scarf",
                "environment_variables": {},
                "withmpi": True,
                "prepend_text": """
                """,
                "append_text": "",
            },
    }),
    "kpoints_mesh": List([1, 1, 1]),
    "code": qe_code,
}

In [6]:
with WorkGraph("EOS_workflow") as wg:

    initial_structure = Path("../structures/NaCl-traj.xyz").resolve()


    final_structures = {}

    for i, struct in enumerate(iread(initial_structure)):
        structure = StructureData(ase=struct)
        
        descriptors_calc = wg.add_task(
            descriptorsCalc,
            **calc_inputs,
            struct=structure,
            calc_per_element=True,
        )

        final_structures[f"structs{i}"] = descriptors_calc.outputs.xyz_output
        
    scales_task = wg.add_task(
        create_scales,
        **scales_inputs,
        structures=final_structures
    )

    qe_task = wg.add_task(
        qe,
        scaled_file=scales_task.outputs.scaled_file,
        **qe_inputs
    )
    
    train_task = wg.add_task(
        create_train_files,
        structures=qe_task.outputs.structures
    )
 

defining outputnode


In [7]:
wg.run()

11/17/2025 04:14:57 PM <165050> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
11/17/2025 04:14:57 PM <165050> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
11/17/2025 04:14:57 PM <165050> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.
11/17/2025 04:14:58 PM <165050> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [10133|WorkGraphEngine|continue_workgraph]: tasks ready to run: Descriptors,Descriptors1,Descriptors2,Descriptors3,Descriptors4,Descriptors5,Descriptors6,Descriptors7,Descriptors8,Descriptors9,Descriptors10,Descriptors11
11/17/2025 04:15:00 PM <165050> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [10133|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 10138, 10143, 10148, 10153, 10158, 10163, 10168, 10173, 10178, 101

{'qe_stress': array([[ 1.86398131e-01,  5.97281145e-17, -1.99093715e-17],
       [ 5.97281145e-17,  1.86398131e-01, -5.97281145e-17],
       [-5.97281145e-17, -1.99093715e-17,  1.86398131e-01]]), 'units': {'energy': 'eV', 'forces': 'ev/Ang', 'stress': 'ev/Ang^3'}, 'qe_energy': -1748.7982484694}
create files: train_file=PosixPath('train.xyz'), valid_file=PosixPath('valid.xyz') and test_file=PosixPath('test.xyz')
Processing: ('all', 'unknown_system'), 12 frames


{}

In [ ]:
#QE task is not appending decriptor so it causes creat_train_to fail